This code reproduces the winning submission for the 2025 Ariel Data Challenge. However, the underlying library has not been cleaned up and is pretty much unreadable. If you're trying to learn from this, I recommend this notebook instead: https://www.kaggle.com/code/jeroencottaar/ariel2025-1st-place-bayesian-exoplanet-analysis

In [1]:
# Load modules
import sys
sys.path.append('/kaggle/input/my-ariel2-library')
import kaggle_support as kgs
import ariel_model

# Load data (just the outline, not the actual sensor readings)
train_data = kgs.load_all_train_data()
test_data = kgs.load_all_test_data()
if len(test_data)==1:
    # If not in submission, run on a few of the training data instead
    data_to_infer = train_data[:20]
else:
    data_to_infer = test_data

# Load the pretrained model
model=kgs.dill_load('/kaggle/input/my-ariel2-model/evaluate_model_Baseline__1099.pickle')[2]
# Alternatively, we could train the model from scratch like this (but too slow for submission):
# model = ariel_model.baseline_model()
# model.train(train_data)
# Note that results would not be 100% the same, since I trained the model in a different environment.

# Some post-training changes
# Use multiple transits when available
model = ariel_model.MultiTransit(model=model)
model.variance_fudge = 1.4 # Increase variance by a bit, since prediction error tends to correlate between transits
model.state = 1
# Allow some planets to fail sanity checks, but raise an error if more than 2
model = ariel_model.SanityWrapper(model=model)
model.run_in_parallel = True
model.state = 1
model.action = 'do_nothing'
model.max_errors = 2
# Increase accuracy of sigma estimation
model.model.model.model.model.model_options.n_samples_sigma_est *= 2

# Make predictions
inferred_data = model.infer(data_to_infer)

# Show score if not submitting
if len(test_data)==1:
    print(kgs.score_metric(inferred_data, data_to_infer))

# Write CSV
kgs.write_submission_csv(kgs.make_submission_dataframe(inferred_data))

kaggle


Processing in parallel:   0%|          | 0/20 [00:00<?, ?it/s]

kaggle
SpawnPoolWorker-3 3
CUDA_VISIBLE_DEVICES= 1


Processing in parallel: 100%|██████████| 20/20 [07:43<00:00, 23.19s/it]


(283,) (283,)
Score:           0.6741
RMS error FGS:   50.13766 ppm
mRMS error FGS:  32.66131 ppm
RMS error AIRS:  51.25211 ppm
mRMS error AIRS: 42.66731 ppm
RMS error AIRSv: 35.40341 ppm
mRMS error AIRSv:27.14927 ppm
(0.6740849318695109, 5.013766203055765e-05, 5.125210789360728e-05)
